# NY Grocery Store Choropleth Maps


## This script contains the following points:
1. Import libraries
2. Import json NYS county mapping data
2. Import new_york_stores.csv dataset
3. Verify the dataset
4. Develop Choropleths of grocery store counts by county
5. Examine counties with less than 100 grocery stores


### 1. Importing Libraries

In [1]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import visualization libraries
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import folium
import json
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### 3. Import NYS county json data
NY State county data pulled from the Cornell University Geospatial Information Repository @ https://cugir.library.cornell.edu/catalog/cugir-007865

In [2]:
# Import ".json" file for NY state by county

nycounty_geo = r'C:\Users\howl6\OneDrive\Certificates\CareerFoundry\Coursework\Data_Immersion\Chapter 6\Food_Desert\02_Data\original_data\cugir-007865-geojson.json'

In [3]:
# reviewing the JSON list for keys

f = open(r"C:\Users\howl6\OneDrive\Certificates\CareerFoundry\Coursework\Data_Immersion\Chapter 6\Food_Desert\02_Data\original_data\cugir-007865-geojson.json",)
  
# returns JSON object as a dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'cugir007865.1', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-74.712396, 44.953008], [-74.711132, 44.953008], [-74.691251, 44.953], [-74.690671, 44.952909], [-74.689482, 44.9529], [-74.668382, 44.952777], [-74.667773, 44.95277], [-74.641872, 44.952621], [-74.636018, 44.910792], [-74.635172, 44.904471], [-74.63121, 44.875391], [-74.63048, 44.869393], [-74.629994, 44.865378], [-74.629519, 44.861096], [-74.629231, 44.859404], [-74.628731, 44.857004], [-74.628559, 44.856249], [-74.626236, 44.838506], [-74.625049, 44.830346], [-74.62294, 44.815154], [-74.622615, 44.812766], [-74.622537, 44.812195], [-74.621928, 44.807683], [-74.619921, 44.792855], [-74.617402, 44.774552], [-74.614223, 44.752374], [-74.613318, 44.74575], [-74.610841, 44.72708], [-74.607362, 44.703708], [-74.604547, 44.686315], [-74.602938, 44.67662], [-74.589474, 44.598732], [-74.571215, 44.470241], [-74.570132, 44.463396], [-74.55863, 44.385142], [-74.557046, 44.372935], [-74.5535, 44.3

### 2. Importing NY stores dataset

In [4]:
path = r'C:\Users\howl6\OneDrive\Certificates\CareerFoundry\Coursework\Data_Immersion\Chapter 6\Food_Desert'

In [5]:
ny_county = pd.read_csv(os.path.join(path, '02_Data', 'cleaned_data', 'new_york_stores.csv'), index_col=False)

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### 4. Verify the dataset

In [7]:
ny_county.head(60)

Unnamed: 0   FIPS State        County  GROC11  GROC16  PCH_GROC_11_16  \
0         1828  36001    NY        Albany      90     101       12.222222   
1         1829  36003    NY      Allegany      12      14       16.666667   
2         1830  36005    NY         Bronx    1115    1296       16.233184   
3         1831  36007    NY        Broome      35      42       20.000000   
4         1832  36009    NY   Cattaraugus      22      19      -13.636364   
5         1833  36011    NY        Cayuga      14      11      -21.428571   
6         1834  36013    NY    Chautauqua      29      30        3.448276   
7         1835  36015    NY       Chemung      13      15       15.384615   
8         1836  36017    NY      Chenango      12       8      -33.333333   
9         1837  36019    NY       Clinton      21      20       -4.761905   
10        1838  36021    NY      Columbia      19      20        5.263158   
11        1839  36023    NY      Cortland       8       7      -12.500000   
12        1840  36025    NY      Delaware      13      12       -7.692308   
13        1841  36027    NY      Dutchess      86      99       15.116279   
14        1842  36029    NY          Erie     247     311       25.910931   
15        1843  36031    NY         Essex      21      19       -9.523810   
16        1844  36033    NY      Franklin      12      13        8.333333   
17        1845  36035    NY        Fulton      10      13       30.000000   
18        1846  36037    NY       Genesee       7      11       57.142857   
19        1847  36039    NY        Greene      14      13       -7.142857   
20        1848  36041    NY      Hamilton       5       4      -20.000000   
21        1849  36043    NY      Herkimer      12      10      -16.666667   
22        1850  36045    NY     Jefferson      25      34       36.000000   
23        1851  36047    NY         Kings    2117    2495       17.855456   
24        1852  36049    NY         Lewis       3       3        0.000000   
25        1853  36051    NY    Livingston      12      13        8.333333   
26        1854  36053    NY       Madison      14      11      -21.428571   
27        1855  36055    NY        Monroe     185     190        2.702703   
28        1856  36057    NY    Montgomery      14      11      -21.428571   
29        1857  36059    NY        Nassau     557     560        0.538600   
30        1858  36061    NY      New York    1333    1319       -1.050263   
31        1859  36063    NY       Niagara      32      34        6.250000   
32        1860  36065    NY        Oneida      55      71       29.090909   
33        1861  36067    NY      Onondaga     125     121       -3.200000   
34        1862  36069    NY       Ontario      18      19        5.555556   
35        1863  36071    NY        Orange     121     108      -10.743802   
36        1864  36073    NY       Orleans      10       8      -20.000000   
37        1865  36075    NY        Oswego      23      21       -8.695652   
38        1866  36077    NY        Otsego      18      18        0.000000   
39        1867  36079    NY        Putnam      33      27      -18.181818   
40        1868  36081    NY        Queens    1410    1624       15.177305   
41        1869  36083    NY    Rensselaer      37      38        2.702703   
42        1870  36085    NY      Richmond     212     242       14.150943   
43        1871  36087    NY      Rockland      93     111       19.354839   
44        1872  36089    NY  St. Lawrence      27      30       11.111111   
45        1873  36091    NY      Saratoga      29      28       -3.448276   
46        1874  36093    NY   Schenectady      40      46       15.000000   
47        1875  36095    NY     Schoharie       4       2      -50.000000   
48        1876  36097    NY      Schuyler       1       2      100.000000   
49        1877  36099    NY        Seneca       9       7      -22.222222   
50        1878  36101    NY       Steuben      20      23       15.000000   


In [8]:
ny_county.tail()

Unnamed: 0   FIPS State       County  GROC11  GROC16  PCH_GROC_11_16  \
57        1885  36115    NY   Washington      10      10        0.000000   
58        1886  36117    NY        Wayne      11      15       36.363636   
59        1887  36119    NY  Westchester     467     485        3.854390   
60        1888  36121    NY      Wyoming      11       8      -27.272727   
61        1889  36123    NY        Yates      10      10        0.000000   

    GROCPTH11  GROCPTH16  PCH_GROCPTH_11_16  SUPERC11  SUPERC16  \
57   0.158559   0.161797           2.041873         0         0   
58   0.117946   0.164973          39.871561         1         2   
59   0.487630   0.495604           1.635209         6         6   
60   0.262818   0.197829         -24.727929         0         0   
61   0.392912   0.399648           1.714491         0         0   

    PCH_SUPERC_11_16  SUPERCPTH11  SUPERCPTH16  PCH_SUPERCPTH_11_16  CONVS11  \
57               0.0     0.000000     0.000000             0.000000       24   
58             100.0     0.010722     0.021996           105.144956       43   
59               0.0     0.006265     0.006131            -2.136819      226   
60               0.0     0.000000     0.000000             0.000000       20   
61               0.0     0.000000     0.000000             0.000000       10   

    CONVS16  PCH_CONVS_11_16  CONVSPTH11  CONVSPTH16  PCH_CONVSPTH_11_16  \
57       25         4.166667    0.380542    0.404491            6.293618   
58       40        -6.976744    0.461062    0.439928           -4.583741   
59      259        14.601770    0.235984    0.264663           12.152937   
60       19        -5.000000    0.477852    0.469843           -1.675857   
61        7       -30.000000    0.392912    0.279754          -28.799856   

    SPECS11  SPECS16  PCH_SPECS_11_16  SPECSPTH11  SPECSPTH16  \
57        3        5        66.666667    0.047568    0.080898   
58        6        3       -50.000000    0.064334    0.032995   
59      149      148        -0.671141    0.155582    0.151236   
60        2        1       -50.000000    0.047785    0.024729   
61        2        2         0.000000    0.078582    0.079930   

    PCH_SPECSPTH_11_16     SNAPS12     SNAPS17  PCH_SNAPS_12_17  SNAPSPTH12  \
57           70.069788   48.000000   52.000000         8.333333    0.762704   
58          -48.713761   49.583333   71.833333        44.873950    0.533372   
59           -2.793619  565.500000  500.000000       -11.582670    0.588040   
60          -48.250451   21.333333   24.166667        13.281250    0.509246   
61            1.714491   19.500000   19.666667         0.854701    0.769413   

    SNAPSPTH17  PCH_SNAPSPTH_12_17  WICS11  WICS16  PCH_WICS_11_16  WICSPTH11  \
57    0.843882           10.643460     8.0     6.0      -25.000000   0.126835   
58    0.792250           48.536143    11.0    11.0        0.000000   0.117968   
59    0.510077          -13.258032   228.0   176.0      -22.807018   0.238428   
60    0.596811           17.195024     6.0     5.0      -16.666666   0.143407   
61    0.788085            2.426830     3.0     2.0      -33.333332   0.117883   

    WICSPTH16  PCH_WICSPTH_11_16  
57   0.097139         -23.412992  
58   0.121207           2.745884  
59   0.181339         -23.943653  
60   0.123744         -13.711248  
61   0.079926         -32.198376

In [9]:
ny_county.shape

(62, 40)

In [10]:
ny_county.columns

Index(['Unnamed: 0', 'FIPS', 'State', 'County', 'GROC11', 'GROC16',
       'PCH_GROC_11_16', 'GROCPTH11', 'GROCPTH16', 'PCH_GROCPTH_11_16',
       'SUPERC11', 'SUPERC16', 'PCH_SUPERC_11_16', 'SUPERCPTH11',
       'SUPERCPTH16', 'PCH_SUPERCPTH_11_16', 'CONVS11', 'CONVS16',
       'PCH_CONVS_11_16', 'CONVSPTH11', 'CONVSPTH16', 'PCH_CONVSPTH_11_16',
       'SPECS11', 'SPECS16', 'PCH_SPECS_11_16', 'SPECSPTH11', 'SPECSPTH16',
       'PCH_SPECSPTH_11_16', 'SNAPS12', 'SNAPS17', 'PCH_SNAPS_12_17',
       'SNAPSPTH12', 'SNAPSPTH17', 'PCH_SNAPSPTH_12_17', 'WICS11', 'WICS16',
       'PCH_WICS_11_16', 'WICSPTH11', 'WICSPTH16', 'PCH_WICSPTH_11_16'],
      dtype='object')

In [11]:
ny_county.dtypes

Unnamed: 0               int64
FIPS                     int64
State                   object
County                  object
GROC11                   int64
GROC16                   int64
PCH_GROC_11_16         float64
GROCPTH11              float64
GROCPTH16              float64
PCH_GROCPTH_11_16      float64
SUPERC11                 int64
SUPERC16                 int64
PCH_SUPERC_11_16       float64
SUPERCPTH11            float64
SUPERCPTH16            float64
PCH_SUPERCPTH_11_16    float64
CONVS11                  int64
CONVS16                  int64
PCH_CONVS_11_16        float64
CONVSPTH11             float64
CONVSPTH16             float64
PCH_CONVSPTH_11_16     float64
SPECS11                  int64
SPECS16                  int64
PCH_SPECS_11_16        float64
SPECSPTH11             float64
SPECSPTH16             float64
PCH_SPECSPTH_11_16     float64
SNAPS12                float64
SNAPS17                float64
PCH_SNAPS_12_17        float64
SNAPSPTH12             float64
SNAPSPTH

### 5. Develop Choropleth of 2016 grocery store counts by county

In [12]:
ny_county['County'] = ny_county['County'].str.title()

In [13]:
# Choropleth of Total Population
data_to_plot = ny_county[['County','GROC16']]

In [14]:
map = folium.Map(location=[41, -74], zoom_start = 5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = nycounty_geo, 
    data = data_to_plot,
    columns = ['County','GROC16'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Grocery 2016").add_to(map)
folium.LayerControl().add_to(map)

map

The largest counties, in particular NY City area counties, are serving as outliers and making it difficult to visual smaller and less urban counties.

In [15]:
# Histogram of total Grocery 2016
sns.histplot(ny_county['GROC16'], bins=50, kde = True)

<AxesSubplot:xlabel='GROC16', ylabel='Count'>

### 6. Examine counties with no more than 100 grocery stores

Eliminating the population of the largest urban areas such as New York City to get a better view of the counties with small populations.

In [16]:
# sort counties by total population with  <= 100,000 as the divider
ny_county.loc[ny_county['GROC16'] <= 100, 'Grocery_flag'] = 'Low_Grocery'

In [17]:
# sort counties by total population between 100,000 and 500,000 as the divider
ny_county.loc[(ny_county ['GROC16'] < 1000) & (ny_county ['GROC16'] > 100), 'Grocery_flag'] = 'Medium_Grocery'

In [18]:
ny_county.loc[ny_county['GROC16'] >= 1000, 'Grocery_flag'] = 'High_Grocery'

In [19]:
ny_county.head(50)

Unnamed: 0   FIPS State        County  GROC11  GROC16  PCH_GROC_11_16  \
0         1828  36001    NY        Albany      90     101       12.222222   
1         1829  36003    NY      Allegany      12      14       16.666667   
2         1830  36005    NY         Bronx    1115    1296       16.233184   
3         1831  36007    NY        Broome      35      42       20.000000   
4         1832  36009    NY   Cattaraugus      22      19      -13.636364   
5         1833  36011    NY        Cayuga      14      11      -21.428571   
6         1834  36013    NY    Chautauqua      29      30        3.448276   
7         1835  36015    NY       Chemung      13      15       15.384615   
8         1836  36017    NY      Chenango      12       8      -33.333333   
9         1837  36019    NY       Clinton      21      20       -4.761905   
10        1838  36021    NY      Columbia      19      20        5.263158   
11        1839  36023    NY      Cortland       8       7      -12.500000   
12        1840  36025    NY      Delaware      13      12       -7.692308   
13        1841  36027    NY      Dutchess      86      99       15.116279   
14        1842  36029    NY          Erie     247     311       25.910931   
15        1843  36031    NY         Essex      21      19       -9.523810   
16        1844  36033    NY      Franklin      12      13        8.333333   
17        1845  36035    NY        Fulton      10      13       30.000000   
18        1846  36037    NY       Genesee       7      11       57.142857   
19        1847  36039    NY        Greene      14      13       -7.142857   
20        1848  36041    NY      Hamilton       5       4      -20.000000   
21        1849  36043    NY      Herkimer      12      10      -16.666667   
22        1850  36045    NY     Jefferson      25      34       36.000000   
23        1851  36047    NY         Kings    2117    2495       17.855456   
24        1852  36049    NY         Lewis       3       3        0.000000   
25        1853  36051    NY    Livingston      12      13        8.333333   
26        1854  36053    NY       Madison      14      11      -21.428571   
27        1855  36055    NY        Monroe     185     190        2.702703   
28        1856  36057    NY    Montgomery      14      11      -21.428571   
29        1857  36059    NY        Nassau     557     560        0.538600   
30        1858  36061    NY      New York    1333    1319       -1.050263   
31        1859  36063    NY       Niagara      32      34        6.250000   
32        1860  36065    NY        Oneida      55      71       29.090909   
33        1861  36067    NY      Onondaga     125     121       -3.200000   
34        1862  36069    NY       Ontario      18      19        5.555556   
35        1863  36071    NY        Orange     121     108      -10.743802   
36        1864  36073    NY       Orleans      10       8      -20.000000   
37        1865  36075    NY        Oswego      23      21       -8.695652   
38        1866  36077    NY        Otsego      18      18        0.000000   
39        1867  36079    NY        Putnam      33      27      -18.181818   
40        1868  36081    NY        Queens    1410    1624       15.177305   
41        1869  36083    NY    Rensselaer      37      38        2.702703   
42        1870  36085    NY      Richmond     212     242       14.150943   
43        1871  36087    NY      Rockland      93     111       19.354839   
44        1872  36089    NY  St. Lawrence      27      30       11.111111   
45        1873  36091    NY      Saratoga      29      28       -3.448276   
46        1874  36093    NY   Schenectady      40      46       15.000000   
47        1875  36095    NY     Schoharie       4       2      -50.000000   
48        1876  36097    NY      Schuyler       1       2      100.000000   
49        1877  36099    NY        Seneca       9       7      -22.222222   

    GROCPTH11  GROCPTH16  PCH_GROCPTH_11_16  SUPERC11  SUPERC16  \
0    0.29

In [20]:
# selecting rows based on 'low grocery' condition

name = ['Low_Grocery']

ny_county2 = ny_county.loc[ny_county['Grocery_flag'].isin(name)]

In [21]:
ny_county2.head(50)

Unnamed: 0   FIPS State        County  GROC11  GROC16  PCH_GROC_11_16  \
1         1829  36003    NY      Allegany      12      14       16.666667   
3         1831  36007    NY        Broome      35      42       20.000000   
4         1832  36009    NY   Cattaraugus      22      19      -13.636364   
5         1833  36011    NY        Cayuga      14      11      -21.428571   
6         1834  36013    NY    Chautauqua      29      30        3.448276   
7         1835  36015    NY       Chemung      13      15       15.384615   
8         1836  36017    NY      Chenango      12       8      -33.333333   
9         1837  36019    NY       Clinton      21      20       -4.761905   
10        1838  36021    NY      Columbia      19      20        5.263158   
11        1839  36023    NY      Cortland       8       7      -12.500000   
12        1840  36025    NY      Delaware      13      12       -7.692308   
13        1841  36027    NY      Dutchess      86      99       15.116279   
15        1843  36031    NY         Essex      21      19       -9.523810   
16        1844  36033    NY      Franklin      12      13        8.333333   
17        1845  36035    NY        Fulton      10      13       30.000000   
18        1846  36037    NY       Genesee       7      11       57.142857   
19        1847  36039    NY        Greene      14      13       -7.142857   
20        1848  36041    NY      Hamilton       5       4      -20.000000   
21        1849  36043    NY      Herkimer      12      10      -16.666667   
22        1850  36045    NY     Jefferson      25      34       36.000000   
24        1852  36049    NY         Lewis       3       3        0.000000   
25        1853  36051    NY    Livingston      12      13        8.333333   
26        1854  36053    NY       Madison      14      11      -21.428571   
28        1856  36057    NY    Montgomery      14      11      -21.428571   
31        1859  36063    NY       Niagara      32      34        6.250000   
32        1860  36065    NY        Oneida      55      71       29.090909   
34        1862  36069    NY       Ontario      18      19        5.555556   
36        1864  36073    NY       Orleans      10       8      -20.000000   
37        1865  36075    NY        Oswego      23      21       -8.695652   
38        1866  36077    NY        Otsego      18      18        0.000000   
39        1867  36079    NY        Putnam      33      27      -18.181818   
41        1869  36083    NY    Rensselaer      37      38        2.702703   
44        1872  36089    NY  St. Lawrence      27      30       11.111111   
45        1873  36091    NY      Saratoga      29      28       -3.448276   
46        1874  36093    NY   Schenectady      40      46       15.000000   
47        1875  36095    NY     Schoharie       4       2      -50.000000   
48        1876  36097    NY      Schuyler       1       2      100.000000   
49        1877  36099    NY        Seneca       9       7      -22.222222   
50        1878  36101    NY       Steuben      20      23       15.000000   
52        1880  36105    NY      Sullivan      32      30       -6.250000   
53        1881  36107    NY         Tioga      11      10       -9.090909   
54        1882  36109    NY      Tompkins      20      21        5.000000   
55        1883  36111    NY        Ulster      47      47        0.000000   
56        1884  36113    NY        Warren      25      27        8.000000   
57        1885  36115    NY    Washington      10      10        0.000000   
58        1886  36117    NY         Wayne      11      15       36.363636   
60        1888  36121    NY       Wyoming      11       8      -27.272727   
61        1889  36123    NY         Yates      10      10        0.000000   

    GROCPTH11  GROCPTH16  PCH_GROCPTH_11_16  SUPERC11  SUPERC16  \
1    0.245887   0.297177          20.859336         0         0   
3    0.175483   0.215573          22.844942         3         3   
4    0.275589   0.244338     

In [23]:
ny_county2.shape

(48, 41)

### My initial questions to explore included identifying the relationship of access to food resources via census tract and if there are relationships to locations such as rural vs. urban, particularly in the upstate New York region; identifying if there is a relationship between population size and access to food resources; and highlighting areas where there are significant gaps in access to food resources.

The choropleths helped to visualize the locations of the various low income, low access tracts at the county level across NYS.  The choropleths suggest a relationship between population size, low income regions, regions with greater SNAP utilization, and access to grocery stores.  

I had been considering focusing on the counties in the Capital Region where I live.  However, the choropleths suggest that gaps are largely absent in this region, and highlight that gaps exist further north, south and west of the Albany region.   As a result, I will be focusing attention on these regions.

In [29]:
# Choropleth of Total Population
data_to_plot = ny_county2[['County','GROC16']]

In [31]:
map = folium.Map(location=[41, -74], zoom_start = 5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = nycounty_geo, 
    data = data_to_plot,
    columns = ['County','GROC16'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Grocery 2016").add_to(map)
folium.LayerControl().add_to(map)

map

In [32]:
# Choropleth of Total Population
data_to_plot = ny_county2[['County','GROC11']]

In [35]:
map = folium.Map(location=[41, -74], zoom_start = 5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = nycounty_geo, 
    data = data_to_plot,
    columns = ['County','GROC11'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Grocery 2011").add_to(map)
folium.LayerControl().add_to(map)

map

In [36]:
ny_county2.to_csv(os.path.join(path, '02_Data', 'cleaned_data', 'NY_low_grocery.csv'))